# AirBnb Price Prediction - Data Cleaning
### Part 1/3

# 1. Introduction
Airbnb is an online platform and marketplace that connects people who want to rent out their properties, such as apartments, houses, or even rooms, with travelers seeking accommodations. It allows individuals to list their properties for short-term rentals and provides a platform for travelers to search and book these accommodations.


Airbnb facilitates the booking process, allowing hosts to create listings with photos, descriptions, and pricing details, while guests can search for available accommodations based on their desired location, travel dates, and other specific criteria. The platform provides a secure payment system, review and rating features, and customer support to ensure a reliable and trustworthy experience for both hosts and guests.


In order to add a new listing, a host has to visit the Airbnb website, register an account and finally add the needed information such as the name and a description of the accommodation, its characteristics (beds, bedrooms, bathrooms, etc..), its positions, the amenities and finally a price.
The host's property price before fees will depend on various things, such as the property location and quality, the amenities it has, and how in demand it is. For this reason, setting the right price is an important operation that can determine the success of Airbnb.


In this project, the goal is to create a price predictor using some Machine Learning techniques. The model will be trained on a dataset containing some characteristics of Airbnb. Finally, the model will be evaluated and tested on a real scenario.


# 2. Dependencies and Tools

Let's start by importing some dependencies. Since the dataset is quite large, I'm going to use Spark, an open-source distributed computing system designed for big data processing and analytics. 

### Install Spark

In [1]:
# Install Spark and related dependencies
!pip install pyspark 

import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
import os

# Create Spark session
conf = SparkConf().\
                set('spark.ui.port', '4050').\
                set('spark.executor.memory', '4G').\
                set('spark.driver.memory', '45G').\
                set('spark.driver.maxResultSize', '10G').\
                set('spark.executor.extraJavaOptions', '-XX:+UseG1GC').\
                setAppName('AirbnbPP1').\
                setMaster('local[*]')

# Create the context
sc = pyspark.SparkContext.getOrCreate(conf=conf)
spark = SparkSession.builder.getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=6dc1fbc87d3fb59631030ad8edfd453f79819a3e6223d7787d4d1526f6d262cc
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [2]:
# Install and configure ngrok
!pip install pyngrok
!ngrok authtoken 2PHrKeMkk5mvdVY3dhwpeV11UP5_7tfznbJvx6L1DSvj8YD5M

# Import ngrok
from pyngrok import ngrok

# Open a ngrok tunnel on the port 4050 where Spark is running
port = '4050'
public_url = ngrok.connect(port).public_url

# Other configurations
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=3d6eb73b236dd2674456eed0d4690ea4e30076d7bbed75f144c78cd2a16fd593
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
print('To access the Spark Web UI console, please click on the following link to the ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}\"'.format(public_url, port))

To access the Spark Web UI console, please click on the following link to the ngrok tunnel "https://6f33-107-167-181-128.ngrok-free.app" -> "http://127.0.0.1:4050"


### Install Libraries

In [4]:
#Install some other useful dependencies
import numpy as np
from numpy import cos, sin, arcsin, sqrt
from math import radians
import json

from pyspark.ml.feature import VectorAssembler, OneHotEncoder, StringIndexer, Imputer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

### Link to Google Drive

In [5]:
# Define GDrive paths
GDRIVE_DIR = '/content/drive'
GDRIVE_HOME_DIR = GDRIVE_DIR + '/MyDrive'
GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + '/Università/BD-Datasets'

In [6]:
# Point Colaboratory our Google Drive
from google.colab import drive

drive.mount(GDRIVE_DIR, force_remount=True)

Mounted at /content/drive


# 3. Dataset
This section aims to import the dataset to Google Colab and read it into a Spark Dataframe.

In [7]:
# Define the dataset name in GDrive
gd_df_name = 'airbnb_aggregated_df.csv'

In [8]:
# Read database into a Spark Dataframe
airbnb_df = spark.read.load(GDRIVE_DATA_DIR + '/' + gd_df_name, 
                           format='csv', 
                           sep=',', 
                           inferSchema='true', 
                           header='true')

In [9]:
print(type(airbnb_df))

<class 'pyspark.sql.dataframe.DataFrame'>


Let's see some information on the dataset to understand if it was uploaded correctly:

In [10]:
# Show if dataset was uploaded correctly
airbnb_df.show(n=5, truncate=False)

+-----------+-------------------------------------+---------+------------+---------------+-------------------------------------------------------------------------+---------+-------------------------------------------+----------+----------------+------------------+------------------+--------------------+-----------------+----------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------+------------------+-------------------+-------------------------+------------------+--------------------+----------------------+---------+--------------+--------+---------+---------------------------+---------------+------------+---------+--------------+--------+----+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# 4. Data Processing
In this section, we will study the dataframe in order to have a better understanding of the features. Then we'll try to change them (if necessary) so that they can be used to build a machine learning model.


### Introduction

First of all, let's print some information on the dataset:

In [11]:
# Print the number of instances and features
print('The database has {:d} instances and {:d} features'.format(airbnb_df.count(), len(airbnb_df.columns)))

The database has 306136 instances and 68 features


In [12]:
# Print the list of features
print('The features are:')
airbnb_df.printSchema()

The features are:
root
 |-- id: double (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: double (nullable = true)
 |-- last_scraped: string (nullable = true)
 |-- source: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_since: string (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_count: integer (nullable = true)
 |-- host_total_listings_count: integer (nullable = true)
 |-- host_verifications: string (nullable = true)
 |-- host_has_profile_pic: string (nullable = true)
 |-- h

### Data Cleaning

It is immediate to notice that there are some features that are useless for the models we are going to create. These features are mainly about ids, urls or other information from which it is not possible to get anything useful. The list of features we can immediatly remove is the following:
* id: id of the accomodation (removed later);
* listing_url: url of the accomodation on the airbnb website;
* scrape_id: id of the scraping operation;
* last_scraped: all dataset were scraped in March 2023 on weekends (Saturday or Sunday) so this information is not interesting;
* source: from were the accomodation was scraped;
* name: name of the accomodation;
* picture_url: url of the picture;
* host_id: id of the host;
* host_url: url of the host webpage;
* host_name: name of the host (not unique);
* host_location and host_neighbourhood: where the host is located (not so interesting information);
* host_thumbnail_url: url of the thumbnail;
* host_picture_url: url of the host profile picture;
* calendar_updated and calendar_last_scraped: again, all dataset was scraped in March 2023 so it is not an interesting information;




In [13]:
# Remove useless features
airbnb_df = airbnb_df.drop(*['listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_location', 'host_neighbourhood', 'host_thumbnail_url', 'host_picture_url', 'calendar_updated', 'calendar_last_scraped'])

I substitute the ID values because the last ones are big integers and difficult to manage. They will be used to merge some columns and it will be removed later:

In [14]:
# Substitute the current (sparse) ids with monotonically increasing ids
airbnb_df = airbnb_df.withColumn('id', monotonically_increasing_id())

Let's print how many null cells we have for each features:

In [15]:
# Print all Null values from the datasets
airbnb_df.agg(*[sum(col(column).isNull().cast('int')).alias(column) for column in airbnb_df.columns]).show()

+---+----------+------------------+------------------+--------------------+-----------------+-------------------+-------------------------+------------------+--------------------+----------------------+----+-------------+--------+---------+-------------+---------+------------+---------+--------------+--------+----+---------+-----+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+---------------+---------------+---------------+----------------+-----------------+---------------------+----------------------+------------+-----------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+----------------+------------------------------+-------------------------------------------+--------------------------------------------+------------------------------

Now, from the list above comes out that there are some features that I must work on. First of all the "bathrooms" features contain all None values so I can delete it immediately (I will get the information on bathrooms with the "bathrooms_text" features - see later).

In [16]:
# Delete "bathrooms" features that contains all NaN values
airbnb_df = airbnb_df.drop('bathrooms')

I also delete "first_review", "last_reviews" and "reviews_per month" since they have too many None values.

In [17]:
# Delete "first_review", "last_review" and "review_per month"
airbnb_df = airbnb_df.drop(*['first_review', 'last_review', 'reviews_per_month'])

In [18]:
# Drop the date columns since they can't be read in the right format
airbnb_df = airbnb_df.drop('host_since')

I also delete the rows that have "host_is_superhost" set to None as well as the other columns where the information on the host is missing. It is not possible to fill the missing values in any ways but using the mode and, in this case, I think this is not a good choice so I rather prefer removing those rows since there are very few:

In [19]:
# Remove rows where the information on the host is missing
airbnb_df = airbnb_df.filter(col('host_is_superhost').isNotNull())
airbnb_df = airbnb_df.filter(col('host_response_time').isNotNull())
airbnb_df = airbnb_df.filter(col('host_response_rate').isNotNull())
airbnb_df = airbnb_df.filter(col('host_acceptance_rate').isNotNull())
airbnb_df = airbnb_df.filter(col('host_listings_count').isNotNull())
airbnb_df = airbnb_df.filter(col('host_total_listings_count').isNotNull())
airbnb_df = airbnb_df.filter(col('host_has_profile_pic').isNotNull())
airbnb_df = airbnb_df.filter(col('host_identity_verified').isNotNull())

For the same reason as before, I also delete the rows that has "bathroom text" set to None:

In [20]:
# Remove rows that has bathroom text set to None
airbnb_df = airbnb_df.filter(col('bathrooms_text').isNotNull())

Now, notice that we also have a bunch of data (more than 60k columns - about 1/5 of the whole dataset) that have no reviews at all. Instead of removing all this data, what I do is substituting the null values with the average of that specific column:

In [21]:
# Compute the average scores
rsr_avg = airbnb_df.agg(avg('review_scores_rating')).first()[0]
rsa_avg = airbnb_df.agg(avg('review_scores_accuracy')).first()[0]
rscl_avg = airbnb_df.agg(avg('review_scores_cleanliness')).first()[0]
rsch_avg = airbnb_df.agg(avg('review_scores_checkin')).first()[0]
rsco_avg = airbnb_df.agg(avg('review_scores_communication')).first()[0]
rsl_avg = airbnb_df.agg(avg('review_scores_location')).first()[0]
rsv_avg = airbnb_df.agg(avg('review_scores_value')).first()[0]

In [22]:
# Substitute the null scores with the averages
airbnb_df = airbnb_df.fillna(rsr_avg, subset=['review_scores_rating'])
airbnb_df = airbnb_df.fillna(rsa_avg, subset=['review_scores_accuracy'])
airbnb_df = airbnb_df.fillna(rscl_avg, subset=['review_scores_cleanliness'])
airbnb_df = airbnb_df.fillna(rsch_avg, subset=['review_scores_checkin'])
airbnb_df = airbnb_df.fillna(rsco_avg, subset=['review_scores_communication'])
airbnb_df = airbnb_df.fillna(rsl_avg, subset=['review_scores_location'])
airbnb_df = airbnb_df.fillna(rsv_avg, subset=['review_scores_value'])

Finally, there is a row with no price. Let's remove it:

In [23]:
# Remove row with no price
airbnb_df = airbnb_df.filter(col('price').isNotNull())

In [24]:
# Print all Null values from the datasets to see the result
airbnb_df.agg(*[sum(col(column).isNull().cast("int")).alias(column) for column in airbnb_df.columns]).show()

+---+------------------+------------------+--------------------+-----------------+-------------------+-------------------------+------------------+--------------------+----------------------+----+-------------+--------+---------+-------------+---------+------------+--------------+--------+----+---------+-----+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+---------------+---------------+---------------+----------------+-----------------+---------------------+----------------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+----------------+------------------------------+-------------------------------------------+--------------------------------------------+-------------------------------------------+
| id|host_response_time|host_re



---



Now, let's try to clean the host information. First of all let's see the content of the columns:

In [25]:
# Print the content of the columns
airbnb_df.select('host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost').show()

+------------------+------------------+--------------------+-----------------+
|host_response_time|host_response_rate|host_acceptance_rate|host_is_superhost|
+------------------+------------------+--------------------+-----------------+
|               N/A|               N/A|                 N/A|                f|
|               N/A|               N/A|                 N/A|                f|
|    within an hour|              100%|                 92%|                t|
|    within an hour|               92%|                100%|                f|
|    within an hour|              100%|                100%|                f|
|    within an hour|              100%|                 99%|                f|
|    within an hour|               98%|                100%|                f|
|within a few hours|               88%|                 94%|                f|
|within a few hours|               98%|                 98%|                f|
|               N/A|               N/A|             

There are still some cells in which we don't have any information ("N/A"). In this case, the missing information probably depends on the fact that the host never replied to any query. In this case I simply leave this field as unknown:

In [26]:
# UTIL: String Indexer function (turn string (categorical) values into numerical ones)
def string_indexer(dataset, input, output):
  indexer = StringIndexer(inputCol=input, outputCol=output)
  encoded_df = indexer.fit(dataset)
  
  for index, label in enumerate(encoded_df.labels):
    print(f"Index: {index}, Label: {label}")

  encoded_df = encoded_df.transform(dataset)
  return encoded_df

In [27]:
# UTIL: Substitute one column with another (input is the column to be substituted, output the one to copy)
def column_substitution(dataset, input, output, type):
  substituted_df = dataset.withColumn(input, dataset[output].cast(IntegerType() if type == 'int' else DoubleType())).drop(output)
  return substituted_df

In [28]:
# Print all unique host_response_time in the dataset
airbnb_df.groupBy('host_response_time').agg(count('*')).show()

+------------------+--------+
|host_response_time|count(1)|
+------------------+--------+
|a few days or more|    5554|
|      within a day|   23752|
|within a few hours|   32707|
|               N/A|   97930|
|    within an hour|  145670|
+------------------+--------+



In [29]:
# Apply One Hot Encoder to turn string (categorical) values into numerical ones
encoded_host_response_time = string_indexer(airbnb_df, 'host_response_time', 'host_response_time_enc')
airbnb_df = column_substitution(encoded_host_response_time, 'host_response_time', 'host_response_time_enc', 'int')

Index: 0, Label: within an hour
Index: 1, Label: N/A
Index: 2, Label: within a few hours
Index: 3, Label: within a day
Index: 4, Label: a few days or more


In [30]:
# Print all unique host_response_rate in the dataset
airbnb_df.groupBy("host_response_rate").agg(count('*')).orderBy('count(1)', ascending=False).show()

+------------------+--------+
|host_response_rate|count(1)|
+------------------+--------+
|              100%|  149742|
|               N/A|   97930|
|               99%|    7196|
|               98%|    6700|
|               90%|    5844|
|                0%|    3411|
|               97%|    2997|
|               80%|    2638|
|               96%|    2419|
|               50%|    2064|
|               94%|    1940|
|               93%|    1680|
|               95%|    1650|
|               92%|    1504|
|               67%|    1368|
|               86%|    1253|
|               91%|    1242|
|               89%|    1221|
|               75%|    1194|
|               83%|    1132|
+------------------+--------+
only showing top 20 rows



Again, we have many unknown values. In this case I substitute the N/A values with the average:

In [31]:
# Select the mean in "host_response_rate" and substitute it to "N/A" values
airbnb_df = airbnb_df.withColumn('host_response_rate', regexp_replace('host_response_rate', '%', '').cast('float'))
host_response_rate_avg = np.round(airbnb_df.select(avg(col('host_response_rate'))).collect()[0][0])
airbnb_df = airbnb_df.fillna(host_response_rate_avg, subset=['host_response_rate'])

# Express the percentages as values between 0 and 1
airbnb_df = airbnb_df.withColumn('host_response_rate', col('host_response_rate') / 100)

In [32]:
# Print all unique host_response_rate in the dataset
airbnb_df.groupBy("host_acceptance_rate").agg(count('*')).orderBy('count(1)', ascending=False).show()

+--------------------+--------+
|host_acceptance_rate|count(1)|
+--------------------+--------+
|                100%|   90902|
|                 N/A|   77756|
|                 99%|   21302|
|                 98%|   10338|
|                 97%|    7796|
|                  0%|    7074|
|                 96%|    5660|
|                 95%|    4708|
|                 94%|    4410|
|                 50%|    3745|
|                 93%|    3592|
|                 92%|    3290|
|                 67%|    2925|
|                 90%|    2602|
|                 91%|    2584|
|                 75%|    2568|
|                 88%|    2565|
|                 86%|    2564|
|                 89%|    2549|
|                 83%|    2517|
+--------------------+--------+
only showing top 20 rows



In [33]:
# Select the mean in "host_acceptance_rate" and substitute it to "N/A" values
airbnb_df = airbnb_df.withColumn('host_acceptance_rate', regexp_replace('host_acceptance_rate', '%', '').cast('float'))
host_acceptance_rate_avg = np.round(airbnb_df.select(avg(col('host_acceptance_rate'))).collect()[0][0])
airbnb_df = airbnb_df.fillna(host_acceptance_rate_avg, subset=['host_acceptance_rate'])

# Express the percentages as values between 0 and 1
airbnb_df = airbnb_df.withColumn('host_acceptance_rate', col('host_acceptance_rate') / 100)

In [34]:
# Apply One Hot Encoder to turn string (categorical) values into numerical ones in "host_is_superhost"
encoded_host_is_superhost = string_indexer(airbnb_df, 'host_is_superhost', 'host_is_superhost_enc')
airbnb_df = column_substitution(encoded_host_is_superhost, 'host_is_superhost', 'host_is_superhost_enc', 'int')

Index: 0, Label: f
Index: 1, Label: t


Let's see the remaining host columns:

In [35]:
# Print the content of the columns
airbnb_df.select('host_listings_count', 'host_total_listings_count', 'host_verifications', 'host_has_profile_pic', 'host_identity_verified').show()

+-------------------+-------------------------+--------------------+--------------------+----------------------+
|host_listings_count|host_total_listings_count|  host_verifications|host_has_profile_pic|host_identity_verified|
+-------------------+-------------------------+--------------------+--------------------+----------------------+
|                  1|                        1|  ['email'; 'phone']|                   t|                     t|
|                  1|                        2|  ['email'; 'phone']|                   t|                     t|
|                  4|                        4|  ['email'; 'phone']|                   t|                     t|
|                  7|                        8|  ['email'; 'phone']|                   t|                     t|
|                  1|                        2|  ['email'; 'phone']|                   t|                     t|
|                  9|                       17|  ['email'; 'phone']|                   t|       

In [36]:
# Apply One Hot Encoder to turn string (categorical) values into numerical ones in "host_has_profile_pic"
encoded_host_has_profile_pic = string_indexer(airbnb_df, 'host_has_profile_pic', 'host_has_profile_pic_enc')
airbnb_df = column_substitution(encoded_host_has_profile_pic, 'host_has_profile_pic', 'host_has_profile_pic_enc', 'int')

Index: 0, Label: t
Index: 1, Label: f


In [37]:
# Print all unique host_response_rate in the dataset
airbnb_df.groupBy("host_identity_verified").agg(count('*')).orderBy('count(1)', ascending=False).show()

+----------------------+--------+
|host_identity_verified|count(1)|
+----------------------+--------+
|                     t|  264817|
|                     f|   40796|
+----------------------+--------+



In [38]:
# Apply One Hot Encoder to turn string (categorical) values into numerical ones in "host_identity_verified"
encoded_host_identity_verified = string_indexer(airbnb_df, 'host_identity_verified', 'host_identity_verified_enc')
airbnb_df = column_substitution(encoded_host_identity_verified, 'host_identity_verified', 'host_identity_verified_enc', 'int')

Index: 0, Label: t
Index: 1, Label: f


Let's finally work on the "host_verifications" column. Here I have a list object that is represented by a string so it is more difficult to manage. Some changes to the object and further analysis must be made to understand how to proceed:

In [39]:
# UTIL: Parse "host_verifications" column
def parse_host_verifications(input):
  input = input.replace('[', '').replace(']', '').replace('\'', '').replace(' ', '').split(';')
  return input

# Define udf_host_verifications function
udf_parse_host_verifications = udf(parse_host_verifications)

In [40]:
# Print unique values in host_verifications column
unique_host_verifications = np.unique(np.concatenate(airbnb_df.select('host_verifications').rdd.map(lambda x: parse_host_verifications(x[0])).collect()), axis = 0, return_counts=True)
print(unique_host_verifications)

(array(['', 'email', 'phone', 'photographer', 'work_email'], dtype='<U12'), array([   632, 280625, 304028,     10,  37658]))


In [41]:
# Remove empty element from unique host verifications and well as "photographer" has few occurrences
host_verifications_to_remove = ['', 'photographer']
host_verifications_mask = np.isin(unique_host_verifications[0], host_verifications_to_remove, invert = True)
unique_host_verifications = unique_host_verifications[0][host_verifications_mask]
print(unique_host_verifications)

['email' 'phone' 'work_email']


In [42]:
# UTIL: Return boolean values depending if an occurence has a certain verification method
def check_verification_method(input):
  input = parse_host_verifications(input)
  return 1 if 'email' in input else 0, 1 if 'phone' in input else 0, 1 if 'work_email' in input else 0

In [43]:
# Let's try to group those values
airbnb_df = airbnb_df.withColumn('is_email_verified', udf(lambda value: check_verification_method(value)[0], IntegerType())('host_verifications')) \
                        .withColumn('is_phone_verified', udf(lambda value: check_verification_method(value)[1], IntegerType())('host_verifications')) \
                        .withColumn('is_work_email_verified', udf(lambda value: check_verification_method(value)[2], IntegerType())('host_verifications'))

In [44]:
# Print the result
airbnb_df.select('host_verifications', 'is_email_verified', 'is_phone_verified', 'is_work_email_verified').show()

+--------------------+-----------------+-----------------+----------------------+
|  host_verifications|is_email_verified|is_phone_verified|is_work_email_verified|
+--------------------+-----------------+-----------------+----------------------+
|  ['email'; 'phone']|                1|                1|                     0|
|  ['email'; 'phone']|                1|                1|                     0|
|  ['email'; 'phone']|                1|                1|                     0|
|  ['email'; 'phone']|                1|                1|                     0|
|  ['email'; 'phone']|                1|                1|                     0|
|  ['email'; 'phone']|                1|                1|                     0|
|  ['email'; 'phone']|                1|                1|                     0|
|           ['phone']|                0|                1|                     0|
|  ['email'; 'phone']|                1|                1|                     0|
|  ['email'; 'ph

In [45]:
# Drop the "host_verifications" column
airbnb_df = airbnb_df.drop('host_verifications')



---



Let's now clean the property information:

In [46]:
# Print the content of the columns
airbnb_df.select('property_type', 'room_type', 'accommodates').show()

+--------------------+---------------+------------+
|       property_type|      room_type|accommodates|
+--------------------+---------------+------------+
|Private room in r...|   Private room|           1|
|Private room in r...|   Private room|           1|
|  Entire rental unit|Entire home/apt|           5|
|Private room in h...|   Private room|           1|
|Private room in r...|   Private room|           2|
|  Entire rental unit|Entire home/apt|           5|
|  Entire rental unit|Entire home/apt|           5|
|  Entire rental unit|Entire home/apt|           3|
|  Entire rental unit|Entire home/apt|           3|
|Private room in c...|   Private room|           2|
|Private room in r...|   Private room|           1|
|Entire serviced a...|Entire home/apt|           4|
|  Entire rental unit|Entire home/apt|           4|
|Private room in r...|   Private room|           1|
|Private room in r...|   Private room|           2|
|  Entire rental unit|Entire home/apt|           5|
|  Entire re

In [47]:
# Print "property_type" unique values
airbnb_df.select('property_type').distinct().collect()

[Row(property_type='Holiday park'),
 Row(property_type='Private room in loft'),
 Row(property_type='Private room in earthen home'),
 Row(property_type='Entire chalet'),
 Row(property_type='Private room in dome'),
 Row(property_type='Earthen home'),
 Row(property_type='Farm stay'),
 Row(property_type='Entire rental unit'),
 Row(property_type='Shared room in tent'),
 Row(property_type='Private room in minsu'),
 Row(property_type='Shared room in hostel'),
 Row(property_type='Shared room'),
 Row(property_type='Private room in condo'),
 Row(property_type='Pension'),
 Row(property_type='Room in boutique hotel'),
 Row(property_type='Private room in pension'),
 Row(property_type='Room in bed and breakfast'),
 Row(property_type='Private room in casa particular'),
 Row(property_type='Entire guesthouse'),
 Row(property_type='Private room in bungalow'),
 Row(property_type='Hut'),
 Row(property_type='Entire cabin'),
 Row(property_type='Shared room in houseboat'),
 Row(property_type='Private room in

In [48]:
# UTIL: Function to group property types together
def transform_property_type(input):
  input_split = input.split(' ')
  if (input_split[0] == 'Entire' or input_split[0] == 'Casa' or input_split[0] == 'Eastern'):
    return 0 #Entire place
  elif (input_split[0] == 'Private' or input_split[0] == 'Room'):
    return 1 #Private room
  elif (input_split[0] == 'Shared'):
    return 2 #Shared room
  else:
    return 3 #Other (e.g. Tent, boat, bus, camper, etc..)

In [49]:
# Let's try to group those values
udf_transform_property_type = udf(transform_property_type)
airbnb_df = airbnb_df.withColumn('property_type', udf_transform_property_type('property_type'))

In [50]:
# Print all unique property_type in the dataset
airbnb_df.groupBy("property_type").agg(count('*')).orderBy('count(1)', ascending=False).show()

+-------------+--------+
|property_type|count(1)|
+-------------+--------+
|            0|  216870|
|            1|   85183|
|            2|    2505|
|            3|    1055|
+-------------+--------+



In [51]:
# Print "room_type" unique values
airbnb_df.select('room_type').distinct().collect()

[Row(room_type='Shared room'),
 Row(room_type='Hotel room'),
 Row(room_type='Entire home/apt'),
 Row(room_type='Private room')]

In [52]:
# Print all unique room_type in the dataset
airbnb_df.groupBy("room_type").agg(count('*')).orderBy('count(1)', ascending=False).show()

+---------------+--------+
|      room_type|count(1)|
+---------------+--------+
|Entire home/apt|  218246|
|   Private room|   82251|
|     Hotel room|    2611|
|    Shared room|    2505|
+---------------+--------+



In [53]:
# Apply One Hot Encoder to turn string (categorical) values into numerical ones in "room_type"
encoded_room_type = string_indexer(airbnb_df, 'room_type', 'room_type_enc')
airbnb_df = column_substitution(encoded_room_type, 'room_type', 'room_type_enc', 'int')

Index: 0, Label: Entire home/apt
Index: 1, Label: Private room
Index: 2, Label: Hotel room
Index: 3, Label: Shared room


It looks like these two features are very similar, we'll eventually study the correlation and see if it is convenient to remove one of them. Let's now work with bathrooms:

In [54]:
# Select unique bathroom strings
airbnb_df.select('bathrooms_text').distinct().collect()

[Row(bathrooms_text='2 shared baths'),
 Row(bathrooms_text='4 baths'),
 Row(bathrooms_text='0 baths'),
 Row(bathrooms_text='4.5 baths'),
 Row(bathrooms_text='17 baths'),
 Row(bathrooms_text='3 baths'),
 Row(bathrooms_text='26 shared baths'),
 Row(bathrooms_text='2.5 baths'),
 Row(bathrooms_text='12 shared baths'),
 Row(bathrooms_text='1.5 shared baths'),
 Row(bathrooms_text='11.5 baths'),
 Row(bathrooms_text='Private half-bath'),
 Row(bathrooms_text='1 shared bath'),
 Row(bathrooms_text='4 shared baths'),
 Row(bathrooms_text='2 baths'),
 Row(bathrooms_text='10 baths'),
 Row(bathrooms_text='8.5 shared baths'),
 Row(bathrooms_text='10.5 baths'),
 Row(bathrooms_text='7 baths'),
 Row(bathrooms_text='24 baths'),
 Row(bathrooms_text='9 shared baths'),
 Row(bathrooms_text='4.5 shared baths'),
 Row(bathrooms_text='1 private bath'),
 Row(bathrooms_text='27 shared baths'),
 Row(bathrooms_text='6 shared baths'),
 Row(bathrooms_text='Half-bath'),
 Row(bathrooms_text='7.5 shared baths'),
 Row(bathr

In [55]:
# UTIL: Function to group bathrooms together
def transform_bathrooms(input):
  input_split = input.split(' ')
  try:
    float(input_split[0])
    if (input_split[1] == 'shared'):
      return input_split[0], 1
    else:
      return input_split[0], 0
  except ValueError:
    return 0.5, 0

In [56]:
# Let's try to group those values
airbnb_df = airbnb_df.withColumn('n_bathrooms', udf(lambda value: transform_bathrooms(value)[0], StringType())('bathrooms_text')) \
                        .withColumn('is_bathroom_shared', udf(lambda value: transform_bathrooms(value)[1], StringType())('bathrooms_text'))

In [57]:
# Show the result
airbnb_df.select('bathrooms_text', 'n_bathrooms', 'is_bathroom_shared').show()

+--------------+-----------+------------------+
|bathrooms_text|n_bathrooms|is_bathroom_shared|
+--------------+-----------+------------------+
|        1 bath|          1|                 0|
| 1 shared bath|          1|                 1|
|     1.5 baths|        1.5|                 0|
|1 private bath|          1|                 0|
| 1 shared bath|          1|                 1|
|        1 bath|          1|                 0|
|        1 bath|          1|                 0|
|        1 bath|          1|                 0|
|        1 bath|          1|                 0|
| 1 shared bath|          1|                 1|
|5 shared baths|          5|                 1|
|        1 bath|          1|                 0|
|        1 bath|          1|                 0|
| 1 shared bath|          1|                 1|
| 1 shared bath|          1|                 1|
|        1 bath|          1|                 0|
|        1 bath|          1|                 0|
|        1 bath|          1|            

In [58]:
# Drop the bathrooms_text column
airbnb_df = airbnb_df.drop('bathrooms_text')

Now, from the table above it is possible to notice that we have about 3500 instances that don't have the number of beds but slightly less than 20k doesn't have the number of rooms. So, instead of removing those rows, it might be the case to try to fill the missing information by making a prediction.

I firstly delete the rows where we don't have the bed information:

In [59]:
# Remove rows where the beds column in None
airbnb_df = airbnb_df.filter(col('beds').isNotNull())

In [60]:
# Print again the number of beds and columns that are set to None to see the result
print("The number of missing beds cells are: " + str(airbnb_df.select(expr("sum(CASE WHEN beds IS NULL THEN 1 ELSE 0 END)")).collect()[0][0]))
print("The number of missing bedrooms cells are: " + str(airbnb_df.select(expr("sum(CASE WHEN bedrooms IS NULL THEN 1 ELSE 0 END)")).collect()[0][0]))

The number of missing beds cells are: 0
The number of missing bedrooms cells are: 22580


Notice that when removing those rows, also the number of empty bedroom cells has decreased meaning that there were rows in which both 'beds' and 'bedrooms' were set to None. Now, we create a simple model to try to predict the missing 'bedroom' values based on the number of beds.

In [61]:
# Prepare the dataset from training the Logistic Regression model
impute_bedrooms_data_notnull = airbnb_df.filter(col('bedrooms').isNotNull()).select('beds', 'bedrooms', 'accommodates') #Not null value for creating the model

impute_bedroom_assembler = VectorAssembler(inputCols=['beds', 'accommodates'], outputCol='features') # Assembled vector with the required features
impute_bedroom_assembled_data_notnull = impute_bedroom_assembler.transform(impute_bedrooms_data_notnull)
impute_bedrooms_train_df, impute_bedrooms_test_df = impute_bedroom_assembled_data_notnull.randomSplit([0.75, 0.25], seed=0) #Split the dataset

impute_bedrooms_train_df = impute_bedrooms_train_df.select(['features', 'bedrooms'])
lr = LogisticRegression(featuresCol='features', labelCol='bedrooms') #Using Logistic Regression for multiclassification prediction

In [62]:
# Model training
lr_model = lr.fit(impute_bedrooms_train_df)

In [63]:
# Test the Logistic Regression model
impute_bedrooms_test_df = impute_bedrooms_test_df.select(['features', 'bedrooms'])
impute_bedrooms_test_df = impute_bedrooms_test_df.withColumnRenamed('bedrooms', 'label')
impute_bedrooms_predictions = lr_model.transform(impute_bedrooms_test_df)
evaluator = MulticlassClassificationEvaluator()
accuracy = evaluator.evaluate(impute_bedrooms_predictions)
print(accuracy)

0.7539359935431424


In [64]:
# Use the model to predict the null values in the bedroom_columns
impute_bedrooms_data_null = airbnb_df.filter(col('bedrooms').isNull()).select('id', 'beds', 'bedrooms', 'accommodates') # Null values to predict
impute_bedrooms_data_null_assembler = impute_bedroom_assembler.transform(impute_bedrooms_data_null)
impute_bedrooms_data_null_assembler = impute_bedrooms_data_null_assembler.select('features')
impute_bedrooms_predictions = lr_model.transform(impute_bedrooms_data_null_assembler) # Prediction vector
impute_bedrooms_predictions = impute_bedrooms_predictions.withColumn('row_id', monotonically_increasing_id()) #Add some id to the prediction column

In [65]:
# Combine the data and fill the null values in the original dataset
impute_bedrooms_joined_data = impute_bedrooms_data_null.withColumn('row_id', monotonically_increasing_id()).drop('bedrooms') #Add some id to the bedroom null data
impute_bedrooms_joined_data = impute_bedrooms_joined_data.join(impute_bedrooms_predictions.withColumnRenamed('prediction', 'bedrooms_new'), on='row_id', how='left_outer') #Merge predictions and null bedroom rows
impute_bedrooms_joined_data = airbnb_df.join(impute_bedrooms_joined_data.select('id', 'bedrooms_new'), on='id', how='left') #Fill missing bedroom in the original dataset
impute_bedrooms_joined_data = impute_bedrooms_joined_data.withColumn('bedrooms_new', coalesce(airbnb_df['bedrooms'], impute_bedrooms_joined_data['bedrooms_new']))
airbnb_df = column_substitution(impute_bedrooms_joined_data, 'bedrooms', 'bedrooms_new', 'int')



---



Now, let's analyze the cities:

In [66]:
# Print all unique cities in the dataset
airbnb_df.select('city').distinct().show()

+--------------------+
|                city|
+--------------------+
| Vila Franca De Xira|
|            Odivelas|
|              Dublin|
|              Berlin|
|              Sintra|
|             Cadaval|
|   Arruda Dos Vinhos|
|            Alenquer|
|             Cascais|
|              Athens|
|                Lyon|
|               Milan|
|Sobral De Monte A...|
|           Barcelona|
|             Amadora|
|           Amsterdam|
|            Brussels|
|            Azambuja|
|               Mafra|
|              Lisboa|
+--------------------+
only showing top 20 rows



As we can see, there are other cities with respect to the ones we want to take into consideration. This is due to the fact that the Lisbon dataset also contains Airbnb in the towns near Lisbon (not only the capital). Since we only want to analyze the capitals, we remove all those instances we are not interested in.

In [67]:
# Remove towns in Lisbon region
airbnb_df = airbnb_df.filter(~col('city').isin(['Vila Franca De Xira', 'Odivelas', 'Sintra', 'Cadaval', 'Arruda Dos Vinhos', 'Alenquer', 'Cascais', 'Sobral De Monte Agrao', 'Amadora', 'Azambuja', 'Mafra', 'Loures', 'Torres Vedras', 'Lourinh', 'Oeiras']))

In [68]:
# Print cities again to see the result
airbnb_df.select('city').distinct().show()

+---------+
|     city|
+---------+
|   Dublin|
|   Berlin|
|   Athens|
|     Lyon|
|    Milan|
|Barcelona|
|Amsterdam|
| Brussels|
|   Lisboa|
|   Munich|
|   Madrid|
|   London|
|    Paris|
|   Vienna|
|     Rome|
+---------+





---



Let's now fix the remaining columns by transforming the binary string values into numerical ones:

In [69]:
# Print all unique room_type in the dataset
airbnb_df.groupBy('has_availability').agg(count('*')).orderBy('count(1)', ascending=False).show()

+----------------+--------+
|has_availability|count(1)|
+----------------+--------+
|               t|  293592|
|               f|    2075|
+----------------+--------+



In [70]:
# Apply One Hot Encoder to turn string (categorical) values into numerical ones in "has_availability"
encoded_has_availability = string_indexer(airbnb_df, 'has_availability', 'has_availability_enc')
airbnb_df = column_substitution(encoded_has_availability, 'has_availability', 'has_availability_enc', 'int')

Index: 0, Label: t
Index: 1, Label: f


In [71]:
# Print all unique room_type in the dataset
airbnb_df.groupBy('instant_bookable').agg(count('*')).orderBy('count(1)', ascending=False).show()

+----------------+--------+
|instant_bookable|count(1)|
+----------------+--------+
|               f|  189172|
|               t|  106495|
+----------------+--------+



In [72]:
# Apply One Hot Encoder to turn string (categorical) values into numerical ones in "instant_bookable"
encoded_instant_bookable = string_indexer(airbnb_df, 'instant_bookable', 'instant_bookable_enc')
airbnb_df = column_substitution(encoded_instant_bookable, 'instant_bookable', 'instant_bookable_enc', 'int')

Index: 0, Label: f
Index: 1, Label: t


Normalize the availability columns so that they that have values between 0 and 1:

In [73]:
# Normalize availability columns
airbnb_df = airbnb_df.withColumn('availability_365', airbnb_df['availability_365']/365)
airbnb_df = airbnb_df.withColumn('availability_90', airbnb_df['availability_90']/90)
airbnb_df = airbnb_df.withColumn('availability_60', airbnb_df['availability_60']/60)
airbnb_df = airbnb_df.withColumn('availability_30', airbnb_df['availability_30']/30)



---



Now, let's try to clean the amenities. As well as the "host_verifications" column, also the "amenities" one is represented by an object that is a list of (different) strings so further considerations have to be made:

In [74]:
# Print the amenities column
airbnb_df.select('amenities').show()

+--------------------+
|           amenities|
+--------------------+
|"[""Coffee maker"...|
|"[""Essentials"";...|
|"[""Coffee maker"...|
|"[""Air condition...|
|"[""Host greets y...|
|"[""Hot water""; ...|
|"[""Oven""; ""Smo...|
|"[""Coffee maker"...|
|"[""Hot water""; ...|
|"[""Washer""; ""W...|
|"[""Smoke alarm""...|
|"[""Hot water""; ...|
|"[""Shampoo""; ""...|
|"[""Shared patio ...|
|"[""Laundromat ne...|
|"[""Shampoo""; ""...|
|"[""City skyline ...|
|"[""Coffee maker"...|
|"[""Coffee maker"...|
|"[""Coffee maker"...|
+--------------------+
only showing top 20 rows



In [75]:
# UTIL: Parse "amenities" column
def parse_amenities(input):
  return [x.strip() for x in input.replace('[', '').replace(']', '').replace('"', '').replace('{','').replace('}', '').replace('\\', '').replace('/', '').replace('-', '').replace(':', '').replace('.', '').replace(',', '').replace('(', '').replace(')', '').replace('*', '').replace('%', '').replace('u2019s', '').replace('u2019n', '').replace('u2013', '').lower().split(';')]

In [76]:
# Print unique values in amenities column
unique_amenities_values, unique_amenities_counts = np.unique(np.concatenate(np.array(airbnb_df.select('amenities').limit(50000).rdd.map(lambda x: parse_amenities(x[0])).collect(), dtype=object)), axis=0, return_counts=True)

In [77]:
# Sort the values to understand which are the most common amenities
sorted_indices = np.argsort(unique_amenities_counts)[::-1]
sorted_values = unique_amenities_values[sorted_indices]
sorted_counts = unique_amenities_counts[sorted_indices]

In [78]:
# Let's print the most common amenities (with more then 5000 occurrences):
for value, count in zip(sorted_values, sorted_counts):
  if count > 5000:
    print(f'{value}: {count}')

kitchen: 45616
wifi: 43867
essentials: 43464
hair dryer: 38917
hangers: 36243
iron: 34303
hot water: 33987
heating: 33980
dishes and silverware: 33078
refrigerator: 31412
washer: 29712
cooking basics: 29622
bed linens: 29074
shampoo: 27527
tv: 26088
elevator: 24310
microwave: 22849
smoke alarm: 19614
oven: 19551
air conditioning: 19521
coffee maker: 18998
dedicated workspace: 18690
dishwasher: 17503
extra pillows and blankets: 15683
long term stays allowed: 15441
stove: 15062
dining table: 14977
cleaning products: 14611
freezer: 14604
hot water kettle: 13830
shower gel: 13712
first aid kit: 12728
host greets you: 12608
drying rack for clothing: 12512
roomdarkening shades: 11998
body soap: 11875
wine glasses: 11869
fire extinguisher: 10360
luggage dropoff allowed: 10225
self checkin: 9764
toaster: 9531
carbon monoxide alarm: 9390
bidet: 9295
pets allowed: 8869
private entrance: 8788
coffee: 8521
baking sheet: 8355
free washer  in unit: 8331
private patio or balcony: 8196
free street par

In the previous lines, I tried to parse the "amenities" column to understand what are the most common amenities in all the accommodations. Unfortunately, this step could only be made using other tools such as Numpy that uses the driver memory. Now, we try to aggregate those values according to some common criteria and finally add new columns to the database that describe if the Airbnb has those amenities:

In [79]:
# Define amenities collection folder name in GDrive
cd_amenities = GDRIVE_DATA_DIR + '/Utils/Amenities/'

In [80]:
# Upload amenities collection from disk
amenities_collection = {}

with open(cd_amenities + 'amenities_collection.json', 'r') as file:
    amenities_collection = json.loads(file.read())

In [81]:
# Print amenities collection that enables to aggregate amenities for category
amenities_collection

{'wifi': 'internet',
 'internet': 'internet',
 'ethernet connection': 'internet',
 'pets allowed': 'pets_allowed',
 'children books and toys': 'children_friendly',
 'children dinnerware': 'children_friendly',
 'crib': 'children_friendly',
 'high chair': 'children_friendly',
 'pack  playtravel crib': 'children_friendly',
 'kitchen': 'cooking_basics',
 'refrigerator': 'cooking_basics',
 'dishes and silverware': 'cooking_basics',
 'cooking basics': 'cooking_basics',
 'microwave': 'cooking_basics',
 'oven': 'cooking_basics',
 'hot water kettle': 'cooking_basics',
 'freezer': 'cooking_basics',
 'toaster': 'cooking_basics',
 'blender': 'cooking_basics',
 'coffee maker': 'cooking_basics',
 'coffee maker nespresso': 'cooking_basics',
 'bbq grill': 'cooking_basics',
 'iron': 'laundry_system',
 'washer': 'laundry_system',
 'dryer': 'laundry_system',
 'free washer  in unit': 'laundry_system',
 'heating': 'heating_cooling_systems',
 'air conditioning': 'heating_cooling_systems',
 'central heating'

In [82]:
# UTIL: Return boolean values depending if an occurence has a certain the verification method
def check_amenities_method(input):
  airbnb_amenities = {'internet': 0, 'pets_allowed': 0, 'children_friendly': 0, 'cooking_basics': 0, 'laundry_system': 0, 'heating_cooling_systems': 0, \
                      'security_devices': 0, 'fireplace': 0, 'tv': 0, 'long_term_stays_allowed': 0, 'self_checkin': 0, 'private_entrance': 0, 'elevator': 0, \
                      'host_greets_you': 0, 'free_parking': 0, 'paid_parking': 0, 'bathtub': 0, 'patio': 0, 'smoking_allowed': 0, 'city_skyline_view': 0, \
                      'breakfast': 0}
  input = parse_amenities(input)
  for amenity in input:
    if amenity in amenities_collection:
      airbnb_amenities[amenities_collection[amenity]] = 1
  return airbnb_amenities['internet'], airbnb_amenities['pets_allowed'], airbnb_amenities['children_friendly'], airbnb_amenities['cooking_basics'], \
  airbnb_amenities['laundry_system'], airbnb_amenities['heating_cooling_systems'], airbnb_amenities['security_devices'], airbnb_amenities['fireplace'], \
  airbnb_amenities['tv'], airbnb_amenities['long_term_stays_allowed'], airbnb_amenities['self_checkin'], airbnb_amenities['private_entrance'], \
  airbnb_amenities['elevator'], airbnb_amenities['host_greets_you'], airbnb_amenities['free_parking'], airbnb_amenities['paid_parking'], \
  airbnb_amenities['bathtub'], airbnb_amenities['patio'], airbnb_amenities['smoking_allowed'], airbnb_amenities['city_skyline_view'], airbnb_amenities['breakfast']

udf_check_amenities_methods = udf(check_amenities_method)

In [83]:
# Let's try to group those values
airbnb_df = airbnb_df.withColumn('has_internet', udf(lambda value: check_amenities_method(value)[0], IntegerType())("amenities")) \
                      .withColumn('is_pet_friendly', udf(lambda value: check_amenities_method(value)[1], IntegerType())("amenities")) \
                      .withColumn('is_children_friendly', udf(lambda value: check_amenities_method(value)[2], IntegerType())("amenities")) \
                      .withColumn('has_cooking_basics', udf(lambda value: check_amenities_method(value)[3], IntegerType())("amenities")) \
                      .withColumn('has_laundry', udf(lambda value: check_amenities_method(value)[4], IntegerType())("amenities")) \
                      .withColumn('has_heating_cooling_systems', udf(lambda value: check_amenities_method(value)[5], IntegerType())("amenities")) \
                      .withColumn('has_security_devices', udf(lambda value: check_amenities_method(value)[6], IntegerType())("amenities")) \
                      .withColumn('has_fireplace', udf(lambda value: check_amenities_method(value)[7], IntegerType())("amenities")) \
                      .withColumn('has_tv', udf(lambda value: check_amenities_method(value)[8], IntegerType())("amenities")) \
                      .withColumn('is_long_term_stays_allowed', udf(lambda value: check_amenities_method(value)[9], IntegerType())("amenities")) \
                      .withColumn('has_self_checkin', udf(lambda value: check_amenities_method(value)[10], IntegerType())("amenities")) \
                      .withColumn('has_private_entrance', udf(lambda value: check_amenities_method(value)[11], IntegerType())("amenities")) \
                      .withColumn('has_elevator', udf(lambda value: check_amenities_method(value)[12], IntegerType())("amenities")) \
                      .withColumn('host_greets_you', udf(lambda value: check_amenities_method(value)[13], IntegerType())("amenities")) \
                      .withColumn('has_free_parking', udf(lambda value: check_amenities_method(value)[14], IntegerType())("amenities")) \
                      .withColumn('has_paid_parking', udf(lambda value: check_amenities_method(value)[15], IntegerType())("amenities")) \
                      .withColumn('has_bathtub', udf(lambda value: check_amenities_method(value)[16], IntegerType())("amenities")) \
                      .withColumn('has_patio', udf(lambda value: check_amenities_method(value)[17], IntegerType())("amenities")) \
                      .withColumn('is_smoking_allowed', udf(lambda value: check_amenities_method(value)[18], IntegerType())("amenities")) \
                      .withColumn('has_city_skyline_view', udf(lambda value: check_amenities_method(value)[19], IntegerType())("amenities")) \
                      .withColumn('has_breakfast', udf(lambda value: check_amenities_method(value)[20], IntegerType())("amenities"))

In [84]:
# Print the resulting columns
amenities_result = airbnb_df.select('id', 'has_internet', 'is_pet_friendly', 'has_cooking_basics', 'has_laundry', 'has_heating_cooling_systems', 'has_security_devices', \
                 'has_fireplace', 'has_tv', 'is_long_term_stays_allowed', 'has_self_checkin', 'has_private_entrance', 'has_elevator', 'host_greets_you', 'has_free_parking', \
                 'has_paid_parking', 'has_bathtub', 'has_patio', 'is_smoking_allowed', 'has_city_skyline_view', 'has_breakfast', 'price')
amenities_result.show()

+---+------------+---------------+------------------+-----------+---------------------------+--------------------+-------------+------+--------------------------+----------------+--------------------+------------+---------------+----------------+----------------+-----------+---------+------------------+---------------------+-------------+-----+
| id|has_internet|is_pet_friendly|has_cooking_basics|has_laundry|has_heating_cooling_systems|has_security_devices|has_fireplace|has_tv|is_long_term_stays_allowed|has_self_checkin|has_private_entrance|has_elevator|host_greets_you|has_free_parking|has_paid_parking|has_bathtub|has_patio|is_smoking_allowed|has_city_skyline_view|has_breakfast|price|
+---+------------+---------------+------------------+-----------+---------------------------+--------------------+-------------+------+--------------------------+----------------+--------------------+------------+---------------+----------------+----------------+-----------+---------+------------------+--

In [85]:
# Drop the "amenities" column
airbnb_df = airbnb_df.drop('amenities')

Finally, price is expressed in the local currency but since we have different currencies (Euro and Sterling Pound) we put everything in the same currency. I choose to express all the prices in US dollars. I convert prices using the exchange rate from late march 2023 (when the dataset was scraped).

In [86]:
# Apply exchange rate to express the price in US dollars
airbnb_df = airbnb_df.withColumn('price', when(airbnb_df['city'] == 'London', airbnb_df['price'] * 1.25).otherwise(airbnb_df['price'] * 1.07))

### Feature Engineering

So far, we didn't deal with the location columns. The information we have about the location of Airbnb are the city, the latitude, the longitude and the neighborhood. The latitude and the longitude features need a more representative value because their values are extremely similar. Also, we have to take into consideration that we deal with different European cities so latitude and longitude might not be the best features to understand if the price of a house changes depending on the location around the city.
In order to increase the accuracy of the predictor, we could add some other features, noticing that the value of an accommodation is usually higher if it is near a station (train, metro), it is in the city center or near a point of interest (e.g. a monument). So I got the coordinates of the stations from Open Street Map and created a Python script to extract all the relevant information (Coordinates of the stations across the different cities) and finally use it to compute the closest station to the Airbnb. The same was done for the city centers and the three best point of interests across all the cities.


In [87]:
# Define the transport coordinates dataset folder name in GDrive
cd_transport = GDRIVE_DATA_DIR + '/Utils/Coordinates/Transport/'

In [88]:
# Open and read the station file
all_transport = {}

with open(cd_transport + 'transport_result.json', 'r', encoding='utf16') as file:
    all_transport = json.loads(file.read())

In [89]:
# Print the stations in each city
print(all_transport)

{'Amsterdam': [{'lat': 52.3696245, 'lon': 4.8926524}, {'lat': 52.3674019, 'lon': 4.9019192}, {'lat': 52.3613436, 'lon': 4.9073455}, {'lat': 52.3302613, 'lon': 4.9560598}, {'lat': 52.3545947, 'lon': 4.9120486}, {'lat': 52.3384028, 'lon': 4.8575769}, {'lat': 52.3726582, 'lon': 4.8352937}, {'lat': 52.3186119, 'lon': 4.9413977}, {'lat': 52.306869, 'lon': 4.9521292}, {'lat': 52.3294483, 'lon': 4.9305189}, {'lat': 52.3317318, 'lon': 4.9176266}, {'lat': 52.3545601, 'lon': 4.9118953}, {'lat': 52.3796134, 'lon': 4.8382223}, {'lat': 52.3122449, 'lon': 4.9470883}, {'lat': 52.3464656, 'lon': 4.9175567}, {'lat': 52.3113175, 'lon': 4.9843223}, {'lat': 52.3230792, 'lon': 4.9735502}, {'lat': 52.3170313, 'lon': 4.9791233}, {'lat': 52.3267669, 'lon': 4.9463398}, {'lat': 52.3235282, 'lon': 4.9364854}, {'lat': 52.3185355, 'lon': 4.9413557}, {'lat': 52.3066107, 'lon': 4.9522364}, {'lat': 52.3524713, 'lon': 4.8343026}, {'lat': 52.328939, 'lon': 4.9662522}, {'lat': 52.3399595, 'lon': 4.9210115}, {'lat': 52.3

In [90]:
# Compute the harvesine distance between two points
# Source: https://stackoverflow.com/questions/29545704/fast-haversine-approximation-python-pandas

def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * arcsin(sqrt(a)) 
    km = 6367 * c
    return float(km)
  
udf_haversine = udf(haversine)

In [91]:
# Compute the smallest distance between a city and an accommodation
def get_smallest_transport_dist(city, latitude, longitude):
  min_distance = None
  for coordinates in all_transport[city]:
    if isinstance(coordinates['lon'], list) or isinstance(coordinates['lat'], list): return False
    if isinstance(latitude, list) or isinstance(longitude, list): return False
    dist = haversine(longitude, latitude, coordinates['lon'], coordinates['lat'])
    if min_distance == None or dist < min_distance: min_distance = dist
  return min_distance

udf_get_smallest_transport_dist = udf(get_smallest_transport_dist)

In [92]:
# Compute all the distances between a station and an Airbnb
airbnb_df = airbnb_df.withColumn('station_dist', udf_get_smallest_transport_dist('city', 'latitude', 'longitude'))

In [93]:
# Print the result
airbnb_df.select('id', 'city', 'station_dist').show()

+---+---------+-------------------+
| id|     city|       station_dist|
+---+---------+-------------------+
| 19|Barcelona|0.05352743051851577|
|  0|Barcelona| 0.3236583626089753|
| 22|Barcelona|0.29048368210771025|
|  7|Barcelona| 0.2224360853927638|
|  6|Barcelona| 0.2726677092891985|
|  9|Barcelona|0.20649558636693452|
| 17|Barcelona| 0.2094955139613898|
|  5|Barcelona|  0.513557906745947|
|  1|Barcelona| 0.4819160257337126|
| 10|Barcelona| 0.2335621330135351|
|  3|Barcelona|0.10307501865020975|
| 12|Barcelona|0.22806826459859553|
|  8|Barcelona|0.37177685011332634|
| 11|Barcelona|0.28692741693538876|
|  2|Barcelona| 0.3782510221722197|
| 18|Barcelona| 0.5183166745596545|
| 14|Barcelona|0.28894351294817616|
| 21|Barcelona| 0.3411357377641502|
| 15|Barcelona|0.33100230416503684|
| 20|Barcelona| 0.2615426882657896|
+---+---------+-------------------+
only showing top 20 rows





---



In [94]:
# Define the transport coordinates dataset folder name in GDrive
cd_city_center = GDRIVE_DATA_DIR + '/Utils/Coordinates/CityCenter'

In [95]:
# Upload the city center coordinates
all_city_centers = {}

with open(cd_city_center+ '/city_center_coordinates.json', 'r') as file:
    all_city_centers = json.loads(file.read())

In [96]:
# Print city center coordinates
print(all_city_centers)

{'Amsterdam': {'lat': 52.37403, 'lon': 4.88969}, 'Lisboa': {'lat': 38.712414, 'lon': -9.138796}, 'Athens': {'lat': 37.98376, 'lon': 23.72784}, 'Brussels': {'lat': 50.84736, 'lon': 4.351756}, 'Berlin': {'lat': 52.51831, 'lon': 13.401192}, 'Rome': {'lat': 41.902226, 'lon': 12.479945}, 'Dublin': {'lat': 53.343264, 'lon': -6.271776}, 'Vienna': {'lat': 48.208339, 'lon': 16.371948}, 'Madrid': {'lat': 40.4165, 'lon': -3.70256}, 'London': {'lat': 51.50853, 'lon': -0.12574}, 'Paris': {'lat': 48.85341, 'lon': 2.3488}, 'Munich': {'lat': 48.13743, 'lon': 11.57549}, 'Milan': {'lat': 45.46427, 'lon': 9.18951}, 'Lyon': {'lat': 45.763372, 'lon': 4.834409}, 'Barcelona': {'lat': 41.387649, 'lon': 2.169269}}


In [97]:
# Define the function to compute the distance between an accommodation in a certain city and the city center
def get_smallest_city_center_dist(city, latitude, longitude):
  city_center = all_city_centers[city]
  return haversine(longitude, latitude, city_center['lon'], city_center['lat'])

udf_get_smallest_city_center_dist = udf(get_smallest_city_center_dist)

In [98]:
# Compute all the distances between a station and an Airbnb
airbnb_df = airbnb_df.withColumn('city_center_dist', udf_get_smallest_city_center_dist('city', 'latitude', 'longitude'))

In [99]:
# Print the result
airbnb_df.select('id', 'city', 'city_center_dist').show()

+---+---------+------------------+
| id|     city|  city_center_dist|
+---+---------+------------------+
| 19|Barcelona|1.5476858178963662|
|  0|Barcelona|2.6390717013664915|
| 22|Barcelona| 0.594702301719104|
|  7|Barcelona|0.3708017748393372|
|  6|Barcelona| 0.916204434422749|
|  9|Barcelona| 3.785549662906064|
| 17|Barcelona|3.8032166584112055|
|  5|Barcelona| 2.842962749739881|
|  1|Barcelona| 5.806555327337496|
| 10|Barcelona| 2.413073800842264|
|  3|Barcelona|0.3195391252561322|
| 12|Barcelona|1.8394439909894282|
|  8|Barcelona|2.6850358411618207|
| 11|Barcelona| 2.536599657908172|
|  2|Barcelona| 1.820609642911876|
| 18|Barcelona|2.1723009263355166|
| 14|Barcelona|1.6333158629411662|
| 21|Barcelona|0.9055668609117927|
| 15|Barcelona|  3.42971601917053|
| 20|Barcelona|1.7036039523388231|
+---+---------+------------------+
only showing top 20 rows





---



In [100]:
# Define the point of interest coordinates dataset folder name in GDrive
cd_poi = GDRIVE_DATA_DIR + '/Utils/Coordinates/POI/'

In [101]:
# Upload the POI coordinates
all_poi = {}

with open(cd_poi + 'poi_coordinates.json', 'r') as file:
    all_poi = json.loads(file.read())

In [102]:
# Print POI coordinates
print(all_poi)

{'Barcelona': [{'name': 'Sagrada Familia', 'lat': 41.49349, 'lon': 2.17434}, {'name': 'Casa Batllo', 'lat': 41.39171, 'lon': 2.16497}, {'name': 'Catedral De Barcelona', 'lat': 41.38396, 'lon': 2.17623}], 'Lyon': [{'name': 'Basilique Notre-Dame De Fourvière', 'lat': 45.76239, 'lon': 4.82191}, {'name': 'Bartholdi Fountain', 'lat': 45.76763, 'lon': 4.83346}, {'name': 'Place Bellecour', 'lat': 45.75777, 'lon': 4.83208}], 'Milan': [{'name': 'Milan Cathedral', 'lat': 45.46411, 'lon': 9.19186}, {'name': 'Teatro alla Scala', 'lat': 45.4674, 'lon': 9.18953}, {'name': 'Castello Sforzesco', 'lat': 45.47039, 'lon': 9.17946}], 'Munich': [{'name': 'Schloss Nymphenburg', 'lat': 48.1584, 'lon': 11.5034}, {'name': 'St. Peter', 'lat': 48.13647, 'lon': 11.57595}, {'name': 'Residenz München', 'lat': 48.14168, 'lon': 11.57972}], 'Amsterdam': [{'name': 'Koninklijk Paleis Amsterdam', 'lat': 52.37282, 'lon': 4.89146}, {'name': 'De Oude Kerk', 'lat': 52.37441, 'lon': 4.8981}], 'Athens': [{'name': 'Parthenon', 

In [103]:
# # Define the function to compute the distance between an accommodation in a certain city and a POI in the same city
def get_smallest_poi_dist(city, latitude, longitude):
  min_distance = None
  for coordinates in all_poi[city]:
    dist = haversine(longitude, latitude, coordinates['lon'], coordinates['lat'])
    if min_distance == None or dist < min_distance: min_distance = dist
  return min_distance

udf_get_smallest_poi_dist = udf(get_smallest_poi_dist)

In [104]:
# Compute all the distances between a point of interest and an Airbnb
airbnb_df = airbnb_df.withColumn('poi_dist', udf_get_smallest_poi_dist('city', 'latitude', 'longitude'))

In [105]:
# Print the result
airbnb_df.select('id', 'city', 'poi_dist').show()

+---+---------+-------------------+
| id|     city|           poi_dist|
+---+---------+-------------------+
| 19|Barcelona|  1.593701279455893|
|  0|Barcelona| 2.7115204745372035|
| 22|Barcelona| 0.1505657737634547|
|  7|Barcelona|0.49924385369762436|
|  6|Barcelona| 0.9071521663811831|
|  9|Barcelona| 3.5562674940971086|
| 17|Barcelona|  3.576283570047511|
|  5|Barcelona|  2.737758559124515|
|  1|Barcelona| 5.4200450264447735|
| 10|Barcelona|  1.836794561979547|
|  3|Barcelona|  0.622088308415851|
| 12|Barcelona| 1.6610042731103665|
|  8|Barcelona| 2.4358057739007912|
| 11|Barcelona| 2.1652610064584015|
|  2|Barcelona| 1.9810562360975634|
| 18|Barcelona| 1.7116053560103535|
| 14|Barcelona| 1.2578511392834482|
| 21|Barcelona|  0.652887512135255|
| 15|Barcelona|  3.209110443165712|
| 20|Barcelona| 0.9988772873261476|
+---+---------+-------------------+
only showing top 20 rows





---



Now, let's finally trasform the cities into numerical values and drop some columns that we don't need anymore:

In [106]:
# Apply One Hot Encoder to turn string (categorical) values into numerical ones in "city"
encoded_room_type = string_indexer(airbnb_df, 'city', 'city_enc')
airbnb_df = column_substitution(encoded_room_type, 'city', 'city_enc', 'int')

Index: 0, Label: London
Index: 1, Label: Paris
Index: 2, Label: Rome
Index: 3, Label: Madrid
Index: 4, Label: Milan
Index: 5, Label: Barcelona
Index: 6, Label: Lisboa
Index: 7, Label: Vienna
Index: 8, Label: Berlin
Index: 9, Label: Athens
Index: 10, Label: Lyon
Index: 11, Label: Dublin
Index: 12, Label: Amsterdam
Index: 13, Label: Brussels
Index: 14, Label: Munich


In [107]:
# Finally remove the information about the neighbourhood, the latitude and longitude
airbnb_df = airbnb_df.drop('neighbourhood', 'latitude', 'longitude')

In [108]:
# Drop the id column since we don't need it anymore
airbnb_df = airbnb_df.drop('id')

In [109]:
# Print the final result
airbnb_df.show()

+------------------+------------------+--------------------+-----------------+-------------------+-------------------------+--------------------+----------------------+----+-------------+---------+------------+--------+----+------------------+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+-------------------+-------------------+--------------------+-------------------+-----------------+---------------------+----------------------+--------------------+----------------------+-------------------------+---------------------+---------------------------+----------------------+-------------------+----------------+------------------------------+-------------------------------------------+--------------------------------------------+-------------------------------------------+-----------------+-----------------+----------------------+-----------+-------------

In [ ]:
# Save the cleaned dataset
airbnb_df.coalesce(1).write.mode('overwrite').csv(GDRIVE_DATA_DIR + '/cleaned_airbnb_df.csv', header=True)